In [1]:
#Replacing a Black-box model by a Global Single Tree Approximation
#A classification case, with both random and deterministic data generation
#Laurent Deborde 2019 march 23th

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

In [3]:
#load dataset
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()
print(cancer.data.shape)

(569, 30)


In [4]:
#build dataframe
import pandas as pd
X = pd.DataFrame(cancer['data'])
y = cancer ['target']
X.describe()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [5]:
# separate between train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target,stratify=cancer.target, random_state=42)

## First let's try to fit a single tree to the data set

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
param_grid = {'max_depth': range(1,10)}

In [7]:
grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid, scoring= 'roc_auc', cv=5,return_train_score=True)
grid_search.fit(X_train, y_train)
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.3f}".format(grid_search.best_score_))
print("Test set score: {:.3f}".format(grid_search.score(X_test, y_test)))

Best parameters: {'max_depth': 2}
Best cross-validation score: 0.933
Test set score: 0.938


#### The optimal max_depth seems to be 2 (quite low). Let's fit a tree of such depth on the whole training set

In [8]:
k=2
tree = DecisionTreeClassifier(max_depth=k)
tree.fit(X_train,y_train)
Acc_appr=tree.score(X_train,y_train)
Acc_test=tree.score(X_test,y_test)
y_pred=tree.predict(X_test)
roc_test=roc_auc_score(y_test, y_pred)
print('for depth',k)
print("accuracy on training set is {:.3f}".format(Acc_appr))
print("accuracy on test set is {:.3f}".format(Acc_test))
print("roc_auc_score on test set is {:.3f}".format(roc_test))

for depth 2
accuracy on training set is 0.946
accuracy on test set is 0.923
roc_auc_score on test set is 0.919


## Let's try a Random forest

In [9]:
param_grid = {'max_depth': [1,2,3,4,5,6,None], 'max_features': [3,5,7,10, None]}
from sklearn.ensemble import RandomForestClassifier
grid_search = GridSearchCV(RandomForestClassifier(), param_grid, scoring= 'roc_auc', cv=5,return_train_score=True)
grid_search.fit(X_train, y_train)
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.3f}".format(grid_search.best_score_))
print("Test set score: {:.3f}".format(grid_search.score(X_test, y_test)))

Best parameters: {'max_depth': 4, 'max_features': 5}
Best cross-validation score: 0.990
Test set score: 0.991


#### Results are much better than with the single tree. Let's fit the RF on the whole training set with the best parameters

In [10]:
rfc = RandomForestClassifier(max_depth= 6, max_features= 5)
np.set_printoptions(precision=3)
rfc.fit(X_train,y_train)
y_pred=rfc.predict(X_test)
roc_test=roc_auc_score(y_test, y_pred)
print("accuracy on training set is {:.3f}".format(rfc.score(X_train,y_train)))
print("accuracy on test set is {:.3f}".format(rfc.score(X_test,y_test)))
print("roc_auc_score on test set is {:.3f}".format(roc_test))

accuracy on training set is 0.993
accuracy on test set is 0.944
roc_auc_score on test set is 0.940


#### Significantly above the single tree results.
### Let's try also a Gradient Boosting

In [11]:
from sklearn.ensemble import GradientBoostingClassifier
param_grid = {'max_depth': [1,3,5,None],'max_features': [1,2,3,5,10,None],'learning_rate' : [0.03,0.1,0.5]}
grid_search = GridSearchCV(GradientBoostingClassifier(), param_grid,scoring= 'roc_auc', cv=5, return_train_score=True)
grid_search.fit(X_train, y_train)
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.3f}".format(grid_search.best_score_))
print("Test set score: {:.3f}".format(grid_search.score(X_test, y_test)))

Best parameters: {'learning_rate': 0.03, 'max_depth': 5, 'max_features': 2}
Best cross-validation score: 0.992
Test set score: 0.993


In [12]:
# let's fit the best parameters on the whole training set
gbr = GradientBoostingClassifier(max_depth=5, max_features=2,learning_rate=0.03)
np.set_printoptions(precision=3)
gbr.fit(X_train,y_train)
y_pred=gbr.predict(X_test)
roc_test=roc_auc_score(y_test, y_pred)
print("accuracy on the training set is {:.3f}".format(gbr.score(X_train,y_train)))
print("accuracy on the test set is {:.3f}".format(gbr.score(X_test,y_test)))
print("roc_auc_score on test set is {:.3f}".format(roc_test))

accuracy on the training set is 1.000
accuracy on the test set is 0.958
roc_auc_score on test set is 0.951


#as an andditionnal measure of prudence, let's try a GB with default parameters : a good result will alleviate fear of overfitting

In [13]:
gbr = GradientBoostingClassifier()
np.set_printoptions(precision=3)
gbr.fit(X_train,y_train)
y_pred=gbr.predict(X_test)
roc_test=roc_auc_score(y_test, y_pred)
print("accuracy on the training set is {:.3f}".format(gbr.score(X_train,y_train)))
print("accuracy on the test set is {:.3f}".format(gbr.score(X_test,y_test)))
print("roc_auc_score on test set is {:.3f}".format(roc_test))

accuracy on the training set is 1.000
accuracy on the test set is 0.958
roc_auc_score on test set is 0.947


## Both RF and GB are significantly better than Single Tree. 

## Q : Let's suppose that for some reasons (need for transparency) we are not allowed to use RF, GB, or other good but complex models. Can we use knowledge from the RF and/or GB to build a better single tree ? 
## A :Yes indeed. It's usually done for the purpose of explanation of complex models. 


## We know that a single tree would have better accuracy if trained on a larger number of data. Let's supplement the original dataset by a synthetic one, for which labels will be predicted by the RF or GB, used as a  kind of oracle.  We'll then try to fit a single tree on this larger data set. 

## First, let's generate new "synthetic" data points. 
### We want them to be numerous, and statistically similar to the original points in the dataset (because we suppose our original data set to be similar to the actual data the model will be used on).
### To do so, we will create them according to a multivariate gaussian distribution with parameters estimated on the original data (A more rigorous approach would be to observe the form of the statistic distribution of the data set and infer from it the distribution shape rather than postulating it's gaussian. An even better strategy if possible would be to use real life unlabelled data). 

#### First let's estimate the parameters of the gaussian distribution on the training set (some feel authorized to also use the test set in such procedure as labels are not used here. I prefer to abstain from it as in real life variables values for data to be labeled are generaly not known when building the model)

In [14]:
X_train= pd.DataFrame(X_train)
echmean=X_train.mean(axis=0)
echcov= X_train.cov()

### Let's draw at random a LARGE number of points according to this distribution. Too large a number will impair computation time, to low a number will impair accuracy. 

In [15]:
X_virt= np.random.multivariate_normal(echmean, echcov, 100000, check_valid ='warn')
d_virt=pd.DataFrame(X_virt, columns=X_train.columns)
d_virt.describe()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,...,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,14.098702,19.295169,91.756108,651.800128,0.096034,0.103642,0.089314,0.048422,0.180263,0.062772,...,16.226747,25.709159,106.992103,875.027629,0.132329,0.256093,0.278126,0.114411,0.291184,0.084052
std,3.519180,4.446081,24.249636,347.062905,0.013356,0.052704,0.081072,0.038583,0.027709,0.007269,...,4.818615,6.306801,33.518099,560.404074,0.022713,0.162484,0.219551,0.067366,0.065595,0.018830
min,-0.281599,-0.140407,-8.731018,-741.775730,0.034892,-0.123739,-0.252489,-0.116072,0.063134,0.028762,...,-4.958019,0.179834,-42.715090,-1632.642562,0.036083,-0.465006,-0.585651,-0.159343,0.014209,-0.005314
25%,11.716777,16.307522,75.354290,417.743091,0.087065,0.068066,0.034279,0.022478,0.161636,0.057900,...,12.987974,21.466966,84.442552,498.235106,0.117078,0.146563,0.129664,0.068884,0.246983,0.071350
50%,14.096291,19.296790,91.731903,650.839715,0.096036,0.103706,0.089293,0.048362,0.180319,0.062779,...,16.219209,25.704349,106.938066,874.847927,0.132365,0.255991,0.277674,0.114428,0.291551,0.084057
75%,16.470124,22.299332,108.119079,886.710120,0.105055,0.139306,0.144352,0.074431,0.198926,0.067646,...,19.487815,29.943834,129.606852,1252.988390,0.147642,0.366100,0.427361,0.160034,0.335305,0.096758
max,30.031063,38.469816,202.348085,2172.685795,0.151025,0.318316,0.427291,0.216790,0.302049,0.095889,...,37.090030,52.019299,258.506553,3393.509676,0.226161,0.946440,1.318122,0.406715,0.560010,0.164923


###  Let's compute the prediction given by our black box oracle (GB with default parameters) on those points. 

In [16]:
y_virt=gbr.predict(d_virt)

### Let's now fit a single tree on this large number of synthetic points (we can/should add the original training set to this synthetic training set, but as the later is much larger than the former it's unlikely to make any significant difference). Warning : computation may be a bit lengthy

In [17]:
 from sklearn.model_selection import cross_val_score
for i in range (4,10):
    tree = DecisionTreeClassifier(max_depth=i,random_state=42)
    score=cross_val_score(tree,d_virt,y_virt,cv=5,scoring= 'roc_auc').mean() 
    print("k=",i,"average cross-validation score: {:.3f}".format(score))    


k= 4 average cross-validation score: 0.983
k= 5 average cross-validation score: 0.987
k= 6 average cross-validation score: 0.990
k= 7 average cross-validation score: 0.992
k= 8 average cross-validation score: 0.991
k= 9 average cross-validation score: 0.986


### Our trees, (especially with the optimal depth of 7) fit well the virtual training set. But do they fit the real data ?

In [18]:
k=7
tree = DecisionTreeClassifier(max_depth=k)
tree.fit(d_virt,y_virt)
Acc_appr=tree.score(X_train,y_train)
Acc_test=tree.score(X_test,y_test)
y_pred=tree.predict(X_test)
roc_test=roc_auc_score(y_test, y_pred)
print('for depth',k)
print("accuracy on training set is {:.3f}".format(Acc_appr))
print("accuracy on test set is {:.3f}".format(Acc_test))
print("roc_auc_score on test set is {:.3f}".format(roc_test))

for depth 7
accuracy on training set is 0.986
accuracy on test set is 0.958
roc_auc_score on test set is 0.951


## Good ! accuracy and roc_auc_score on test set are significantly above the initial tree scores (albeit at the price of increased complexity/depth). Still they are under the GB scores, which was expected.

### Let's try with Random Forest now

In [19]:
z_virt=rfc.predict(d_virt)

In [20]:
for i in range (5,11):
    tree1 = DecisionTreeClassifier(max_depth=i,random_state=42)
    score=cross_val_score(tree1,d_virt,z_virt,scoring= 'roc_auc',cv=5).mean() 
    print("k=",i,"average cross-validation score: {:.3f}".format(score))

k= 5 average cross-validation score: 0.996
k= 6 average cross-validation score: 0.996
k= 7 average cross-validation score: 0.996
k= 8 average cross-validation score: 0.993
k= 9 average cross-validation score: 0.988
k= 10 average cross-validation score: 0.980


In [21]:
# again, let's try this tree on the original set
k=5
tree1 = DecisionTreeClassifier(max_depth=k)
tree1.fit(d_virt,y_virt)
Acc_appr=tree1.score(X_train,y_train)
Acc_test=tree1.score(X_test,y_test)
y_pred=tree1.predict(X_test)
roc_test=roc_auc_score(y_test, y_pred)
print('for depth',k)
print("accuracy on training set is {:.3f}".format(Acc_appr))
print("accuracy on test set is {:.3f}".format(Acc_test))
print("roc_auc_score on test set is {:.3f}".format(roc_test))

for depth 5
accuracy on training set is 0.986
accuracy on test set is 0.944
roc_auc_score on test set is 0.944


### Again, significant improvement over the initial tree. 

### For the sake of illustration, let's now try a different strategy to generate artificial data. Rather than gaussian random draws, we will use a deterministic procedure : take the middle point between each pair of initial data

In [22]:
long=len(X_train)
court=len(X_train.columns)
X_virt=np.zeros((long*long,court))
X_val=X_train.values

In [23]:
for i in range(long):
    for j in range(long):
        for k in range(court):
            X_virt[i+long*j,k]=(X_val[i,k]+X_val[j,k])/2

In [24]:
d_virt=pd.DataFrame(X_virt, columns=X_train.columns)
d_virt.describe()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
count,181476.000000,181476.000000,181476.000000,181476.000000,181476.000000,181476.000000,181476.000000,181476.000000,181476.000000,181476.000000,...,181476.000000,181476.000000,181476.000000,181476.000000,181476.000000,181476.000000,181476.000000,181476.000000,181476.000000,181476.000000
mean,14.075202,19.295047,91.592911,649.627230,0.096018,0.103460,0.088931,0.048211,0.180203,0.062779,...,16.202542,25.708263,106.803732,872.524413,0.132365,0.256041,0.277706,0.114156,0.291210,0.084079
std,2.478274,3.141389,17.081338,244.329577,0.009426,0.037197,0.057164,0.027193,0.019525,0.005111,...,3.396074,4.456901,23.624039,394.905197,0.015997,0.114667,0.154931,0.047498,0.046171,0.013242
min,6.981000,9.710000,43.790000,143.500000,0.062510,0.019380,0.000000,0.000000,0.106000,0.049960,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,12.255000,17.075000,78.935000,468.187500,0.089445,0.075935,0.043310,0.025865,0.166850,0.059140,...,13.635000,22.535000,88.855000,577.000000,0.121400,0.172050,0.161449,0.077410,0.260650,0.074765
50%,13.705000,19.020000,88.955000,590.725000,0.095680,0.098025,0.078720,0.044595,0.178600,0.062025,...,15.565000,25.435000,102.650000,761.850000,0.131908,0.236850,0.255250,0.111263,0.285025,0.081785
75%,15.680000,21.230000,102.560000,796.400000,0.102185,0.124925,0.122825,0.064300,0.191650,0.065595,...,18.350000,28.545000,121.450000,1087.300000,0.142700,0.316500,0.371146,0.145890,0.313800,0.090865
max,28.110000,39.280000,188.500000,2499.000000,0.142500,0.345400,0.426400,0.191300,0.304000,0.097440,...,33.130000,49.540000,229.300000,3432.000000,0.218400,1.058000,1.252000,0.291000,0.663800,0.207500


In [25]:
#let's label those points with the black box prediction
y_virt=gbr.predict(d_virt)

In [26]:
#as above, let(s fit a tree on those data
for i in range (4,10):
    tree = DecisionTreeClassifier(max_depth=i,random_state=42)
    score=cross_val_score(tree,d_virt,y_virt,cv=5,scoring= 'roc_auc').mean() 
    print("k=",i,"average cross-validation score: {:.3f}".format(score))    

k= 4 average cross-validation score: 0.992
k= 5 average cross-validation score: 0.994
k= 6 average cross-validation score: 0.996
k= 7 average cross-validation score: 0.997
k= 8 average cross-validation score: 0.998
k= 9 average cross-validation score: 0.997


In [27]:
#hour of truth : let's look at the predicting ability of this new tree : 
k=8
tree = DecisionTreeClassifier(max_depth=k)
tree.fit(d_virt,y_virt)
Acc_appr=tree.score(X_train,y_train)
Acc_test=tree.score(X_test,y_test)
y_pred=tree.predict(X_test)
roc_test=roc_auc_score(y_test, y_pred)
print('for depth',k)
print("accuracy on training set is {:.3f}".format(Acc_appr))
print("accuracy on test set is {:.3f}".format(Acc_test))
print("roc_auc_score on test set is {:.3f}".format(roc_test))

for depth 8
accuracy on training set is 0.991
accuracy on test set is 0.951
roc_auc_score on test set is 0.949


### Quite good ! 